In [ ]:
COLAB = False
if COLAB : 
    !pip install tensorflow-addons
    from google.colab import drive
    drive.mount('/content/gdrive')

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import pandas as pd
import numpy as np
import shutil
import os

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import Model

from plotly import express as px

import logging
logging.getLogger('tensorflow_addons').setLevel(logging.ERROR)

In [ ]:
if COLAB : 
    DATA_PATH = '/content/gdrive/MyDrive/Projects/BleauGuessr/data/fulldata/'
else :
    DATA_PATH = '../data/fulldata/'

folders = next(os.walk(DATA_PATH))[1]  # Get a list of all subdirectories
NUM_CLASSES = len(folders)  # Count the number of subdirectories

IMG_SIZE = (512, 512)

## Define data pipeline

In [ ]:
dataset_train, dataset_test = image_dataset_from_directory(
                    DATA_PATH,
                    labels='inferred',
                    label_mode='categorical',
                    class_names=None,
                    color_mode='rgb',
                    batch_size=4,
                    image_size=IMG_SIZE,
                    shuffle=True,
                    seed=0,
                    subset='both',
                    validation_split=0.4,
                    interpolation='bilinear',
                    follow_links=False,
                    crop_to_aspect_ratio=True,
                )

class_names = dataset_train.class_names

In [ ]:
dataset_test.class_names

# Augmentations

In [ ]:
# Define the preprocessing function
def _normalize_img(img, label):
    img = tf.cast(img, tf.float32) / 255.
    return (img, label)


# Define the preprocessing layers for data augmentation
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomRotation(0.1), 
  tf.keras.layers.RandomZoom(0.2, 0.5),
  tf.keras.layers.RandomContrast(0.4),
  tf.keras.layers.RandomBrightness(0.4, value_range=(0,1)),
])

# Normalize the images
dataset_train = dataset_train.map(_normalize_img)
dataset_test = dataset_test.map(_normalize_img)

# Augmentations only on the training set
dataset_train = dataset_train.map(lambda x, y: (data_augmentation(x), y))
dataset_train = dataset_train.prefetch(tf.data.AUTOTUNE) # Speeds up training by caching augmentations and dataset in memory


# Define model

In [ ]:
# Base ResNet model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=IMG_SIZE +(3,))

# Add top embedding layer
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(128, 
                 activation='relu', 
                 )(x)
x = layers.Dense(NUM_CLASSES,
                 activation='softmax',)(x)
model = Model(inputs=base_model.input, outputs=x)


# Freeze all but last conv layer of base model
for layer in base_model.layers : 
    if layer.name not in ['conv5_block3_3_conv'] : 
        layer.trainable = False

In [ ]:
# # Define the input shape of the images
# input_shape = IMG_SIZE + (3,)

# # Define the convolutional neural network architecture
# model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu', input_shape=IMG_SIZE +(3,)),
#     tf.keras.layers.MaxPooling2D(pool_size=2),
#     tf.keras.layers.Dropout(0.3),
#     tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'),
#     tf.keras.layers.MaxPooling2D(pool_size=2),
#     tf.keras.layers.Conv2D(filters=128, kernel_size=2, padding='same', activation='relu'),
#     tf.keras.layers.MaxPooling2D(pool_size=2),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
# ])

In [ ]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'],
              )

# Train

In [ ]:
# Train the model
history = model.fit(dataset_train, epochs=20, validation_data=dataset_test)

# Predict

In [ ]:
ds_test = [exs for exs in dataset_test.as_numpy_iterator()]
len(ds_test)

In [ ]:
for exs in dataset_test.as_numpy_iterator() : 
    
    X,Y = exs

    for x,y in zip(X,Y) : 

        print('='*20)

        label = class_names[np.argmax(y)]

        fig = px.imshow(x)
        fig.show()

        img = np.expand_dims(x, axis=0) # add batch dimension
        res = model.predict(img)

        print(f'truth : {label}')
        print(f'predict : {class_names[np.argmax(res)]}')

        fig2 = px.bar(x=class_names, y=res[0])
        fig2.show()

In [ ]:
X.shape